# EDA

### Business Problem
Understanding how players engage with the PlayStation ecosystem to improve retention and drive revenue (game sales, PlayStation Plus subscriptions).

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Load the datasets
activity_df = pd.read_csv('synthetic_playstation_activity_realistic.csv')
plus_df = pd.read_csv('synthetic_playstation_plus_realistic.csv')
users_df = pd.read_csv('synthetic_playstation_users_realistic.csv')

View the dataframes

In [22]:
activity_df.head()

,user_id,game_played,genre,playtime_minutes,session_start
0,PSN_00001,Metal Gear Solid,Stealth,45,2024-07-24 02:36:28
1,PSN_00001,Horizon Zero Dawn,Action RPG,43,2025-03-19 11:48:00
2,PSN_00001,Ghost of Tsushima,Action-Adventure,33,2024-04-07 15:23:40
3,PSN_00001,Red Dead Redemption 2,Action-Adventure,33,2024-11-05 11:13:41
4,PSN_00001,God of War,Action-Adventure,104,2025-03-08 07:40:30


In [32]:
plus_df

,user_id,plus_tier,subscription_start,renewal_count,last_renewal_date
0,PSN_00001,Premium,2024-11-30 07:54:37,2,2024-12-30 06:17:31
1,PSN_00002,Premium,2024-05-12 02:02:24,2,2024-07-12 12:04:05
2,PSN_00003,Extra,2025-02-16 13:01:02,1,2025-02-16 13:01:02
3,PSN_00004,Essential,2024-04-25 21:57:49,1,2024-05-28 14:05:13
4,PSN_00005,Extra,2023-04-22 10:50:39,1,2023-04-22 10:50:39
...,...,...,...,...,...
9995,PSN_09996,NaN,NaN,0,NaN
9996,PSN_09997,Essential,2023-10-11 04:55:03,0,NaN
9997,PSN_09998,Extra,2024-04-19 14:41:24,1,2024-04-19 14:41:24
9998,PSN_09999,Essential,2025-01-01 02:34:49,3,2025-04-11 22:03:41


In [26]:
users_df.head()

,user_id,signup_date,region
0,PSN_00001,2024-03-01 15:52:58,Oceania
1,PSN_00002,2022-09-08 15:12:05,Europe
2,PSN_00003,2024-02-09 09:13:37,Europe
3,PSN_00004,2022-05-16 18:10:10,Oceania
4,PSN_00005,2022-04-13 10:36:24,Asia


In [120]:
users_df['signup_date'] = pd.to_datetime(users_df['signup_date'])
activity_df['session_start'] = pd.to_datetime(activity_df['session_start'])
plus_df['subscription_start'] = pd.to_datetime(plus_df['subscription_start'])
plus_df['last_renewal_date'] = pd.to_datetime(plus_df['last_renewal_date'])

In [30]:
print("Missing values in activity_df:\n", activity_df.isnull().sum())
print("\nMissing values in plus_df:\n", plus_df.isnull().sum())
print("\nMissing values in users_df:\n", users_df.isnull().sum())

Missing values in activity_df:
 user_id             0
game_played         0
genre               0
playtime_minutes    0
session_start       0
dtype: int64

Missing values in plus_df:
 user_id                  0
plus_tier             3044
subscription_start    3044
renewal_count            0
last_renewal_date     3803
dtype: int64

Missing values in users_df:
 user_id        0
signup_date    0
region         0
dtype: int64


In plus_df, NaN values exists for users without PlayStation Plus and there are more NaN in the column "last_renewal_date" because some Plus members choose to not renew their subscriptions. We can interpret NaN values as "No Subscription."

### Feature Engineering

In [118]:
# Creating user tenure (in days)
now = datetime.now()
users_df['tenure_days'] = (now - users_df['signup_date']).dt.days

users_df

,user_id,signup_date,region,tenure_days
0,PSN_00001,2024-03-01 15:52:58,Oceania,403
1,PSN_00002,2022-09-08 15:12:05,Europe,943
2,PSN_00003,2024-02-09 09:13:37,Europe,425
3,PSN_00004,2022-05-16 18:10:10,Oceania,1058
4,PSN_00005,2022-04-13 10:36:24,Asia,1092
...,...,...,...,...
9995,PSN_09996,2023-02-09 02:54:12,Oceania,790
9996,PSN_09997,2022-06-16 05:55:49,Asia,1028
9997,PSN_09998,2023-03-31 14:55:54,Asia,739
9998,PSN_09999,2024-08-16 08:03:07,Europe,236


Now, I want to combine all the dataframes to make it easier for analysis.

In [66]:
# Left merging on all users
merged_df = pd.merge(users_df, plus_df, on='user_id', how='left')

merged_df

,user_id,signup_date,region,tenure_days,plus_tier,subscription_start,renewal_count,last_renewal_date
0,PSN_00001,2024-03-01 15:52:58,Oceania,403,Premium,2024-11-30 07:54:37,2,2024-12-30 06:17:31
1,PSN_00002,2022-09-08 15:12:05,Europe,943,Premium,2024-05-12 02:02:24,2,2024-07-12 12:04:05
2,PSN_00003,2024-02-09 09:13:37,Europe,425,Extra,2025-02-16 13:01:02,1,2025-02-16 13:01:02
3,PSN_00004,2022-05-16 18:10:10,Oceania,1058,Essential,2024-04-25 21:57:49,1,2024-05-28 14:05:13
4,PSN_00005,2022-04-13 10:36:24,Asia,1092,Extra,2023-04-22 10:50:39,1,2023-04-22 10:50:39
...,...,...,...,...,...,...,...,...
9995,PSN_09996,2023-02-09 02:54:12,Oceania,790,NaN,NaN,0,NaN
9996,PSN_09997,2022-06-16 05:55:49,Asia,1028,Essential,2023-10-11 04:55:03,0,NaN
9997,PSN_09998,2023-03-31 14:55:54,Asia,739,Extra,2024-04-19 14:41:24,1,2024-04-19 14:41:24
9998,PSN_09999,2024-08-16 08:03:07,Europe,236,Essential,2025-01-01 02:34:49,3,2025-04-11 22:03:41


# Calculating Metrics and Extracting Insights

In [75]:
# Overall PlayStation Plus Subscription Rate
plus_subscription_rate = (merged_df['plus_tier'].notna().sum() / len(merged_df)) * 100
print(f"\nOverall PlayStation Plus Subscription Rate: {plus_subscription_rate:.2f}%")


Overall PlayStation Plus Subscription Rate: 69.56%


In [104]:
# Churn Rate Based on Last Renewal
churned_users = merged_df[merged_df['plus_tier'].notna() & merged_df['last_renewal_date'].isnull()]
total_plus_users = merged_df['plus_tier'].notna().sum()
churn_rate = (len(churned_users) / total_plus_users) * 100 if total_plus_users > 0 else 0
print(f"Simplified Churn Rate (based on no last renewal): {churn_rate:.2f}%")

Simplified Churn Rate (based on no last renewal): 10.91%


In [79]:
# Average Playtime per User
average_playtime_per_user = activity_df.groupby('user_id')['playtime_minutes'].mean().reset_index()
merged_df = pd.merge(merged_df, average_playtime_per_user, on='user_id', how='left')
print("\nAverage Playtime per User (first 5):\n", merged_df[['user_id', 'playtime_minutes']].head())


Average Playtime per User (first 5):
      user_id  playtime_minutes
0  PSN_00001         55.689441
1  PSN_00002         64.132948
2  PSN_00003         60.784314
3  PSN_00004         59.679487
4  PSN_00005         72.679245


In [81]:
# Most Popular Games
most_popular_games = activity_df['game_played'].value_counts().nlargest(10)
print("\nTop 10 Most Popular Games:\n", most_popular_games)


Top 10 Most Popular Games:
 game_played
Fortnite                 105362
GTA V                    100058
Marvel's Spider-Man       96973
Red Dead Redemption 2     93583
God of War                89959
The Last of Us            87044
Ghost of Tsushima         87035
Elden Ring                84707
Uncharted                 80851
Assassin's Creed          77971
Name: count, dtype: int64


In [83]:
# Subscription Rate by Region
subscription_by_region = merged_df.groupby('region')['plus_tier'].apply(lambda x: x.notna().sum() / len(x) * 100).sort_values(ascending=False)
print("\nPlayStation Plus Subscription Rate by Region:\n", subscription_by_region)


PlayStation Plus Subscription Rate by Region:
 region
Latin America    71.854472
Asia             69.569472
Oceania          69.068451
North America    68.895643
Europe           68.446602
Name: plus_tier, dtype: float64


In [97]:
# Impact of Tenure on Subscription
bins = [0, 30, 90, 365, 10000]
labels = ['0-30 Days', '31-90 Days', '91-365 Days', '>365 Days']
merged_df['tenure_group'] = pd.cut(merged_df['tenure_days'], bins=bins, labels=labels, right=False)
subscription_by_tenure = merged_df.groupby('tenure_group')['plus_tier'].apply(lambda x: x.notna().sum() / len(x) * 100).sort_values(ascending=False)
print("\nPlayStation Plus Subscription Rate by Tenure Group:\n", subscription_by_tenure)


PlayStation Plus Subscription Rate by Tenure Group:
 tenure_group
0-30 Days      74.683544
31-90 Days     70.476190
91-365 Days    69.520817
>365 Days      69.321578
Name: plus_tier, dtype: float64


C:\Users\jaeng\AppData\Local\Temp\ipykernel_14488\12404248.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  subscription_by_tenure = merged_df.groupby('tenure_group')['plus_tier'].apply(lambda x: x.notna().sum() / len(x) * 100).sort_values(ascending=False)


In [99]:
merged_df

,user_id,signup_date,region,tenure_days,plus_tier,subscription_start,renewal_count,last_renewal_date,playtime_minutes,tenure_group
0,PSN_00001,2024-03-01 15:52:58,Oceania,403,Premium,2024-11-30 07:54:37,2,2024-12-30 06:17:31,55.689441,>365 Days
1,PSN_00002,2022-09-08 15:12:05,Europe,943,Premium,2024-05-12 02:02:24,2,2024-07-12 12:04:05,64.132948,>365 Days
2,PSN_00003,2024-02-09 09:13:37,Europe,425,Extra,2025-02-16 13:01:02,1,2025-02-16 13:01:02,60.784314,>365 Days
3,PSN_00004,2022-05-16 18:10:10,Oceania,1058,Essential,2024-04-25 21:57:49,1,2024-05-28 14:05:13,59.679487,>365 Days
4,PSN_00005,2022-04-13 10:36:24,Asia,1092,Extra,2023-04-22 10:50:39,1,2023-04-22 10:50:39,72.679245,>365 Days
...,...,...,...,...,...,...,...,...,...,...
9995,PSN_09996,2023-02-09 02:54:12,Oceania,790,NaN,NaN,0,NaN,59.600000,>365 Days
9996,PSN_09997,2022-06-16 05:55:49,Asia,1028,Essential,2023-10-11 04:55:03,0,NaN,65.148936,>365 Days
9997,PSN_09998,2023-03-31 14:55:54,Asia,739,Extra,2024-04-19 14:41:24,1,2024-04-19 14:41:24,66.684685,>365 Days
9998,PSN_09999,2024-08-16 08:03:07,Europe,236,Essential,2025-01-01 02:34:49,3,2025-04-11 22:03:41,75.972222,91-365 Days


In [108]:
# Save the DataFrame for Tableau
final_df_for_tableau = merged_df.copy()
final_df_for_tableau.to_csv('playstation_analysis_for_tableau.csv', index=False)